<a href="https://colab.research.google.com/github/patrol114/patrol114/blob/main/Kopia_notatnika_KuCoin_Advanced_Analysis_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import asyncio
import logging
import os
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Any
import json
from dataclasses import dataclass, field
from enum import Enum
import threading
import time
from collections import deque
import math
import statistics

# Główne biblioteki
import pandas as pd
import numpy as np
try:
    import talib
    TALIB_AVAILABLE = True
except ImportError:
    TALIB_AVAILABLE = False
    print("Warning: TA-Lib not available. Using simplified indicators.")

# Import ccxt (powrót do ccxt)
try:
    import ccxt.async_support as ccxt
    CCXT_AVAILABLE = True
except ImportError:
    CCXT_AVAILABLE = False
    print("Error: 'ccxt' library is required. Install it using 'pip install ccxt[async]' or 'pip install ccxt'.")

import colorama
from colorama import Fore, Back, Style
from rich.console import Console
from rich.table import Table
from rich.panel import Panel
from rich.progress import Progress, SpinnerColumn, TextColumn
from rich.live import Live
from rich.layout import Layout
from rich.text import Text
from rich.align import Align
from rich.columns import Columns
from rich.box import ROUNDED
from rich.rule import Rule
from rich.tree import Tree
from rich.status import Status
from rich.bar import Bar
from rich.prompt import Prompt
from rich.markdown import Markdown
from dotenv import load_dotenv
import warnings
warnings.filterwarnings('ignore')

# Ładowanie konfiguracji z .env
load_dotenv()

# Inicjalizacja kolorów
colorama.init(autoreset=True)
console = Console(width=int(os.getenv('CONSOLE_WIDTH', 120)))

# Konfiguracja logowania
logging.basicConfig(
    level=getattr(logging, os.getenv('LOG_LEVEL', 'INFO')),
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(os.getenv('LOG_FILE', 'kucoin_analysis.log')),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

class TrendDirection(Enum):
    STRONG_BULLISH = "🚀 Silnie Wzrostowy"
    BULLISH = "📈 Wzrostowy"
    NEUTRAL = "⚖️ Neutralny"
    BEARISH = "📉 Spadkowy"
    STRONG_BEARISH = "💥 Silnie Spadkowy"

class SignalStrength(Enum):
    VERY_STRONG = "💪 Bardzo Silny"
    STRONG = "🔥 Silny"
    MODERATE = "⚡ Umiarkowany"
    WEAK = "🌊 Słaby"
    VERY_WEAK = "💨 Bardzo Słaby"

class MarketSentiment(Enum):
    EXTREME_FEAR = "😱 Ekstremiczny Strach"
    FEAR = "😰 Strach"
    NEUTRAL = "😐 Neutralny"
    GREED = "🤑 Chciwość"
    EXTREME_GREED = "🤯 Ekstremalna Chciwość"

@dataclass
class PredictionModel:
    timeframe: str
    prediction_price: float
    confidence: float
    probability_up: float
    probability_down: float
    key_levels: List[float]
    expected_volatility: float
    trend_strength: float
    volume_prediction: float
    risk_reward_ratio: float

@dataclass
class PriceTarget:
    level: float
    probability: float
    timeframe: str
    resistance_support: str
    description: str
    risk_level: str = "Medium"

@dataclass
class CandlestickPattern:
    name: str
    strength: int
    bullish: bool
    description: str
    reliability: float

@dataclass
class TechnicalSignal:
    indicator: str
    signal: str
    strength: float
    timeframe: str
    description: str

@dataclass
class MarketAnalysis:
    symbol: str
    current_price: float
    price_change_24h: float
    volume_change_24h: float
    trend: TrendDirection
    signal_strength: SignalStrength
    market_sentiment: MarketSentiment
    confidence: float
    volatility: float
    volume_analysis: str
    key_levels: List[float]
    price_targets: List[PriceTarget]
    risk_assessment: str
    market_structure: str
    candlestick_patterns: List[CandlestickPattern]
    technical_signals: List[TechnicalSignal]
    divergences: List[str]
    predictions: Dict[str, PredictionModel]
    fibonacci_levels: Dict[str, float]
    elliott_wave_count: str
    market_sentiment_score: float
    fear_greed_impact: str
    liquidity_analysis: str
    order_flow_prediction: str
    support_resistance_levels: Dict[str, List[float]]
    trend_analysis: Dict[str, Any]
    momentum_indicators: Dict[str, float]
    volume_profile: Dict[str, float]
    time_based_analysis: Dict[str, Any]

@dataclass
class WatchlistInstrument:
    symbol: str
    name: str
    current_price: float
    change_24h: float
    volume_24h: float
    market_cap: Optional[float] = None
    analysis: Optional[MarketAnalysis] = None
    last_update: datetime = field(default_factory=datetime.now)
    active_position: bool = False
    position_size: float = 0.0
    entry_price: float = 0.0
    unrealized_pnl: float = 0.0

class SimpleIndicators:
    """Klasa z prostymi wskaźnikami technicznymi gdy TA-Lib nie jest dostępny"""

    @staticmethod
    def sma(data, period):
        """Simple Moving Average"""
        return data.rolling(window=period).mean()

    @staticmethod
    def ema(data, period):
        """Exponential Moving Average"""
        return data.ewm(span=period).mean()

    @staticmethod
    def rsi(data, period=14):
        """Relative Strength Index"""
        delta = data.diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
        rs = gain / loss
        return 100 - (100 / (1 + rs))

    @staticmethod
    def macd(data, fast=12, slow=26, signal=9):
        """MACD"""
        ema_fast = SimpleIndicators.ema(data, fast)
        ema_slow = SimpleIndicators.ema(data, slow)
        macd_line = ema_fast - ema_slow
        signal_line = SimpleIndicators.ema(macd_line, signal)
        histogram = macd_line - signal_line
        return macd_line, signal_line, histogram

    @staticmethod
    def bollinger_bands(data, period=20, std_dev=2):
        """Bollinger Bands"""
        sma = SimpleIndicators.sma(data, period)
        std = data.rolling(window=period).std()
        upper_band = sma + (std * std_dev)
        lower_band = sma - (std * std_dev)
        return upper_band, sma, lower_band

    @staticmethod
    def atr(high, low, close, period=14):
        """Average True Range"""
        tr1 = high - low
        tr2 = abs(high - close.shift())
        tr3 = abs(low - close.shift())
        true_range = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
        return true_range.rolling(window=period).mean()

class AdvancedKuCoinAnalyzer:
    def __init__(self):
        """Inicjalizacja zaawansowanego systemu analizy"""
        # Sprawdzamy dostępność ccxt
        if not CCXT_AVAILABLE:
            raise ImportError("Required library 'ccxt' is not installed or configured correctly.")

        self.config = self._load_config()
        self.watchlist: List[WatchlistInstrument] = []
        self.is_running = False
        self.market_data_cache = {}
        self.analysis_history = deque(maxlen=1000)
        self.prediction_history = deque(maxlen=500)
        self.price_history = {}
        self.pattern_history = {}
        self.exchange = None # Klient ccxt

        # Inicjalizacja klienta ccxt
        asyncio.run(self._init_client()) # _init_client jest teraz asynchroniczny

        # Inicjalizacja watchlisty
        asyncio.run(self._init_watchlist())

        # Statystyki predykcji
        self.prediction_stats = {
            'total_predictions': 0,
            'correct_predictions': 0,
            'accuracy_by_timeframe': {},
            'accuracy_by_instrument': {},
            'best_patterns': {},
            'signal_performance': {}
        }

        # Wyświetlenie bannera
        self._show_banner()

    def _load_config(self) -> Dict:
        """Ładowanie konfiguracji z zmiennych środowiskowych"""
        return {
            'kucoin': {
                'api_key': os.getenv('KUCOIN_API_KEY'),
                'api_secret': os.getenv('KUCOIN_API_SECRET'),
                'api_passphrase': os.getenv('KUCOIN_API_PASSPHRASE'),
                'sandbox': os.getenv('KUCOIN_SANDBOX', 'true').lower() == 'true'
            },
            'analysis': {
                'refresh_interval': int(os.getenv('ANALYSIS_REFRESH_INTERVAL', 30)),
                'max_instruments': int(os.getenv('MAX_INSTRUMENTS_DISPLAY', 15)),
                'prediction_horizon': int(os.getenv('PREDICTION_HORIZON_HOURS', 24)),
                'confidence_threshold': float(os.getenv('CONFIDENCE_THRESHOLD', 0.7))
            },
            'display': {
                'use_colors': os.getenv('USE_COLORS', 'true').lower() == 'true',
                'show_detailed': os.getenv('SHOW_DETAILED_ANALYSIS', 'true').lower() == 'true',
                'show_predictions': os.getenv('SHOW_PREDICTIONS', 'true').lower() == 'true',
                'show_patterns': os.getenv('SHOW_CANDLESTICK_PATTERNS', 'true').lower() == 'true'
            },
            'technical': {
                'rsi_period': int(os.getenv('RSI_PERIOD', 14)),
                'macd_fast': int(os.getenv('MACD_FAST', 12)),
                'macd_slow': int(os.getenv('MACD_SLOW', 26)),
                'macd_signal': int(os.getenv('MACD_SIGNAL', 9)),
                'bb_period': int(os.getenv('BOLLINGER_BANDS_PERIOD', 20)),
                'bb_std': float(os.getenv('BOLLINGER_BANDS_STD', 2)),
                'atr_period': int(os.getenv('ATR_PERIOD', 14)),
                'volume_ma_period': int(os.getenv('VOLUME_MA_PERIOD', 20))
            },
            'alerts': {
                'price_change_threshold': float(os.getenv('PRICE_CHANGE_THRESHOLD', 0.05)),
                'volume_spike_threshold': float(os.getenv('VOLUME_SPIKE_THRESHOLD', 2.0)),
                'volatility_threshold': float(os.getenv('VOLATILITY_THRESHOLD', 0.03))
            },
            'prediction': {
                'weight_volume': float(os.getenv('WEIGHT_VOLUME', 0.3)),
                'weight_technical': float(os.getenv('WEIGHT_TECHNICAL', 0.4)),
                'weight_momentum': float(os.getenv('WEIGHT_MOMENTUM', 0.3)),
                'weight_pattern': float(os.getenv('WEIGHT_PATTERN', 0.2)),
                'weight_sentiment': float(os.getenv('WEIGHT_SENTIMENT', 0.1))
            }
        }

    async def _init_client(self):
        """Inicjalizacja klienta ccxt KuCoin Futures API"""
        try:
            kucoin_config = self.config['kucoin']

            if not all([kucoin_config.get('api_key'),
                         kucoin_config.get('api_secret'),
                         kucoin_config.get('api_passphrase')]):
                console.print("[yellow]⚠️ Brak kluczy API - uruchamianie w trybie demo[/yellow]")
                self.demo_mode = True
                self.exchange = None
                return

            self.exchange = ccxt.kucoinfutures({
                'apiKey': kucoin_config['api_key'],
                'secret': kucoin_config['api_secret'],
                'password': kucoin_config['api_passphrase'],
                'options': {
                    'defaultType': 'future', # Ustawienie dla KuCoin Futures
                },
                'enableRateLimit': True, # Włączenie limitowania zapytań
                'sandbox': kucoin_config['sandbox'] # Użycie sandboxa, jeśli skonfigurowano
            })

            console.print("[green]✅ Klient ccxt KuCoin Futures zainicjalizowany[/green]")
            console.print("[green]🔑 Łączę się z KuCoin Futures…[/green]") # Dodany log
            self.demo_mode = False

        except Exception as e:
            console.print(f"[red]❌ Błąd inicjalizacji klienta ccxt: {e}[/red]")
            self.demo_mode = True
            self.exchange = None
            logger.error(f"Błąd inicjalizacji klienta ccxt: {e}")

    async def _init_watchlist(self):
        """Inicjalizacja watchlisty z popularnymi instrumentami i aktualnymi danymi"""
        instruments_config = [
            ('BTC/USDT', 'Bitcoin', 45000),
            ('ETH/USDT', 'Ethereum', 3000),
            ('BNB/USDT', 'Binance Coin', 300),
            ('ADA/USDT', 'Cardano', 0.5),
            ('SOL/USDT', 'Solana', 80),
            ('DOT/USDT', 'Polkadot', 8),
            ('AVAX/USDT', 'Avalanche', 25),
            ('LINK/USDT', 'Chainlink', 15),
            ('MATIC/USDT', 'Polygon', 1.2),
            ('ATOM/USDT', 'Cosmos', 12),
            ('NEAR/USDT', 'Near Protocol', 4),
            ('UNI/USDT', 'Uniswap', 8),
            ('LTC/USDT', 'Litecoin', 100),
            ('XRP/USDT', 'Ripple', 0.6),
            ('APT/USDT', 'Aptos', 10)
        ]

        if not self.demo_mode and self.exchange:
            try:
                # Pobieranie otwartych pozycji z API REST - jako snapshot początkowy
                positions = await self.exchange.fetch_positions()
                console.print(f"[green]✅ Pozycje pobrane: {positions}[/green]") # Dodany log
                position_map = {pos['symbol']: pos for pos in positions if pos.get('contracts', 0) != 0}

                for symbol, name, base_price in instruments_config:
                    current_price = base_price
                    active_position = False
                    position_size = 0.0
                    entry_price = 0.0
                    unrealized_pnl = 0.0

                    # Pobieranie aktualnej ceny z REST API jako początkowej
                    try:
                        ticker = await self.exchange.fetch_ticker(symbol)
                        current_price = float(ticker.get('last', base_price))
                        change_24h = float(ticker.get('percentage', 0.0))
                        volume_24h = float(ticker.get('quoteVolume', 0.0))
                    except Exception as e:
                        logger.error(f"Błąd pobierania ceny dla {symbol} (ccxt): {e}")
                        console.print(f"[yellow]⚠️ Używam ceny domyślnej dla {symbol}: {base_price}[/yellow]")
                        change_24h = 0.0
                        volume_24h = 0.0

                    # Sprawdzanie pozycji
                    if symbol in position_map:
                        pos = position_map[symbol]
                        active_position = True
                        position_size = float(pos.get('contracts', 0.0))
                        entry_price = float(pos.get('entryPrice', 0.0))
                        unrealized_pnl = float(pos.get('unrealisedPnl', 0.0))

                    instrument = WatchlistInstrument(
                        symbol=symbol,
                        name=name,
                        current_price=current_price,
                        change_24h=change_24h,
                        volume_24h=volume_24h,
                        active_position=active_position,
                        position_size=position_size,
                        entry_price=entry_price,
                        unrealized_pnl=unrealized_pnl
                    )

                    self.watchlist.append(instrument)

            except Exception as e:
                logger.error(f"Błąd pobierania pozycji lub danych początkowych z REST API (ccxt): {e}")
                console.print("[yellow]⚠️ Błąd pobierania danych z REST API, używam trybu demo dla watchlisty[/yellow]")
                self._init_watchlist_fallback(instruments_config)
        else:
            self._init_watchlist_fallback(instruments_config)

    def _init_watchlist_fallback(self, instruments_config: List[tuple]):
        """Zapasowa inicjalizacja watchlisty w trybie demo"""
        for symbol, name, base_price in instruments_config:
            instrument = WatchlistInstrument(
                symbol=symbol,
                name=name,
                current_price=base_price,
                change_24h=0.0,
                volume_24h=0.0,
                active_position=np.random.choice([True, False], p=[0.3, 0.7])
            )

            if instrument.active_position:
                instrument.position_size = np.random.uniform(0.1, 2.0)
                instrument.entry_price = base_price * np.random.uniform(0.95, 1.05)
                instrument.unrealized_pnl = (instrument.current_price - instrument.entry_price) * instrument.position_size

            self.watchlist.append(instrument)

    def _show_banner(self):
        """Wyświetlenie bannera systemowego"""
        banner = """
╔══════════════════════════════════════════════════════════════════════════════════════╗
║             🚀 KuCoin Advanced Analysis & Prediction System 🚀                       ║
║                         📊 Wersja 4.0 - Ulepszona 📊                                  ║
║                                                                                      ║
║ 🔮 Zaawansowana analiza techniczna i prognozowanie ruchów cenowych                   ║
║ 📈 Wykrywanie wzorców świecowych i sygnałów technicznych                             ║
║ 🎯 Predykcja celów cenowych i poziomów wsparcia/oporu                                 ║
║ ⚡ Analiza momentum i struktur rynkowych                                              ║
║ 🌊 Analiza przepływu wolumenu i sentimentu rynkowego                                  ║
║                                                                                      ║
║ ⚠️ UWAGA: System służy wyłącznie do analizy - nie otwiera pozycji!                   ║
╚══════════════════════════════════════════════════════════════════════════════════════╝
        """
        console.print(Panel(banner, style="bold cyan", box=ROUNDED))

    async def get_market_data(self, symbol: str, timeframe: str, limit: int = 500) -> pd.DataFrame:
        """Pobieranie danych rynkowych (OHLCV) z cache'owaniem lub generowanie demo"""
        cache_key = f"{symbol}_{timeframe}"

        if cache_key in self.market_data_cache:
            cached_data, timestamp = self.market_data_cache[cache_key]
            if datetime.now() - timestamp < timedelta(minutes=5): # Odśwież co 5 minut
                return cached_data

        try:
            if not self.demo_mode and self.exchange:
                # ccxt używa standardowych nazw interwałów ('1m', '5m', '1h', '1d' itd.)
                # więc mapowanie jest uproszczone

                ohlcv = await self.exchange.fetch_ohlcv(
                    symbol=symbol,
                    timeframe=timeframe,
                    since=None, # ccxt radzi sobie z 'since' automatycznie, jeśli limit jest podany
                    limit=limit
                )

                if ohlcv:
                    df = pd.DataFrame(ohlcv, columns=['time', 'open', 'high', 'low', 'close', 'volume'])
                    df['time'] = pd.to_datetime(df['time'], unit='ms')
                    df = df.set_index('time')

                    for col in ['open', 'high', 'low', 'close', 'volume']:
                        df[col] = df[col].astype(float)

                    self.market_data_cache[cache_key] = (df, datetime.now())
                    return df
                else:
                    logger.warning(f"Brak danych OHLCV dla {symbol} {timeframe} z API. Generowanie danych demo.")
                    df = self._generate_realistic_data(symbol, timeframe, limit)
                    self.market_data_cache[cache_key] = (df, datetime.now())
                    return df

            else: # Tryb demo lub brak klienta
                df = self._generate_realistic_data(symbol, timeframe, limit)
                self.market_data_cache[cache_key] = (df, datetime.now())
                return df

        except Exception as e:
            logger.error(f"Błąd pobierania danych OHLCV dla {symbol}: {e}. Generowanie danych demo.")
            return self._generate_realistic_data(symbol, timeframe, limit)

    def _generate_realistic_data(self, symbol: str, timeframe: str, limit: int) -> pd.DataFrame:
        """Generowanie realistycznych danych testowych z trendem"""
        np.random.seed(hash(symbol + timeframe) % 2**32)

        base_prices = {
            'BTC/USDT': 45000, 'ETH/USDT': 3000, 'BNB/USDT': 300,
            'ADA/USDT': 0.5, 'SOL/USDT': 80, 'DOT/USDT': 8,
            'AVAX/USDT': 25, 'LINK/USDT': 15, 'MATIC/USDT': 1.2,
            'ATOM/USDT': 12, 'NEAR/USDT': 4, 'UNI/USDT': 8,
            'LTC/USDT': 100, 'XRP/USDT': 0.6, 'APT/USDT': 10
        }

        base_price = base_prices.get(symbol, 100)

        freq_map = {
            '1m': '1min', '5m': '5min', '15m': '15min', '30m': '30min',
            '1h': '1H', '2h': '2H', '4h': '4H', '1d': '1D', '1w': '1W'
        }
        freq = freq_map.get(timeframe, '15min')

        end_time = datetime.now()
        dates = pd.date_range(end=end_time, periods=limit, freq=freq)

        volatility = {
            '1m': 0.008, '5m': 0.012, '15m': 0.018, '30m': 0.025,
            '1h': 0.035, '4h': 0.055, '1d': 0.08, '1w': 0.15
        }.get(timeframe, 0.02)

        trend_strength = np.random.choice([-0.3, -0.1, 0, 0.1, 0.3], p=[0.2, 0.2, 0.2, 0.2, 0.2])

        prices = []
        volumes = []
        current_price = base_price

        for i in range(limit):
            trend_component = trend_strength * 0.001
            cycle_component = 0.01 * np.sin(i * 0.05) * volatility
            noise_component = np.random.normal(0, volatility)
            mean_reversion = -0.1 * (current_price - base_price) / base_price

            total_change = trend_component + cycle_component + noise_component + mean_reversion
            current_price *= (1 + total_change)

            intraday_range = abs(np.random.normal(0, volatility * 0.5))

            open_price = current_price * (1 + np.random.normal(0, 0.003))
            close_price = current_price

            if close_price > open_price:
                high_price = max(open_price, close_price) * (1 + intraday_range * 0.7)
                low_price = min(open_price, close_price) * (1 - intraday_range * 0.3)
            else:
                high_price = max(open_price, close_price) * (1 + intraday_range * 0.3)
                low_price = min(open_price, close_price) * (1 - intraday_range * 0.7)

            base_volume = np.random.lognormal(8, 1)
            volume_multiplier = 1 + (abs(total_change) * 5)
            volume = base_volume * volume_multiplier

            prices.append({
                'open': open_price,
                'high': high_price,
                'low': low_price,
                'close': close_price
            })
            volumes.append(volume)

        df = pd.DataFrame(prices, index=dates)
        df['volume'] = volumes

        return df

    def calculate_advanced_indicators(self, df: pd.DataFrame) -> pd.DataFrame:
        """Obliczanie kompletnego zestawu wskaźników technicznych"""
        try:
            df = df.copy()

            if not TALIB_AVAILABLE:
                df['rsi'] = SimpleIndicators.rsi(df['close'])
                df['macd'], df['macd_signal'], df['macd_hist'] = SimpleIndicators.macd(df['close'])
                df['upper_bb'], df['middle_bb'], df['lower_bb'] = SimpleIndicators.bollinger_bands(df['close'])
                df['atr'] = SimpleIndicators.atr(df['high'], df['low'], df['close'])
                return df

            df['rsi'] = talib.RSI(df['close'], timeperiod=self.config['technical']['rsi_period'])
            df['macd'], df['macd_signal'], df['macd_hist'] = talib.MACD(
                df['close'],
                fastperiod=self.config['technical']['macd_fast'],
                slowperiod=self.config['technical']['macd_slow'],
                signalperiod=self.config['technical']['macd_signal']
            )
            df['upper_bb'], df['middle_bb'], df['lower_bb'] = talib.BBANDS(
                df['close'],
                timeperiod=self.config['technical']['bb_period'],
                nbdevup=self.config['technical']['bb_std'],
                nbdevdn=self.config['technical']['bb_std']
            )
            df['atr'] = talib.ATR(
                df['high'],
                df['low'],
                df['close'],
                timeperiod=self.config['technical']['atr_period']
            )

            # Dodatkowe wskaźniki
            df['adx'] = talib.ADX(df['high'], df['low'], df['close'], timeperiod=14)
            df['cci'] = talib.CCI(df['high'], df['low'], df['close'], timeperiod=14)
            df['stoch_k'], df['stoch_d'] = talib.STOCH(
                df['high'], df['low'], df['close'],
                fastk_period=14, slowk_period=3, slowd_period=3
            )
            df['obv'] = talib.OBV(df['close'], df['volume'])
            df['vwap'] = (df['volume'] * (df['high'] + df['low'] + df['close']) / 3).cumsum() / df['volume'].cumsum()

            # Wolumen MA
            df['volume_ma'] = df['volume'].rolling(window=self.config['technical']['volume_ma_period']).mean()

            return df
        except Exception as e:
            logger.error(f"Błąd podczas obliczania wskaźników technicznych: {e}")
            return df # Zwróć oryginalny DataFrame w przypadku błędu

    def detect_candlestick_patterns(self, df: pd.DataFrame) -> List[CandlestickPattern]:
        """Wykrywanie wzorców świecowych za pomocą TA-Lib"""
        patterns = []
        if not TALIB_AVAILABLE or df.empty or len(df) < 30: # Potrzeba wystarczająco dużo danych
            return patterns

        open_prices = df['open'].values
        high_prices = df['high'].values
        low_prices = df['low'].values
        close_prices = df['close'].values

        # Lista wzorców TA-Lib do sprawdzenia
        talib_patterns = {
            "CDLDOJI": {"name": "Doji", "bullish": False, "desc": "Wskazuje na niezdecydowanie na rynku."},
            "CDLHAMMER": {"name": "Młot", "bullish": True, "desc": "Potencjalne odwrócenie trendu spadkowego."},
            "CDLINVERTEDHAMMER": {"name": "Odwrócony Młot", "bullish": True, "desc": "Potencjalne odwrócenie trendu spadkowego."},
            "CDLMORNINGSTAR": {"name": "Gwiazda Poranna", "bullish": True, "desc": "Silny sygnał odwrócenia trendu spadkowego."},
            "CDLENGULFING": {"name": "Formacja Objęcia", "bullish": None, "desc": "Wskazuje na zmianę sentymentu. Sprawdź, czy jest bycza czy niedźwiedzia."},
            "CDLPIERCING": {"name": "Formacja Przebicia", "bullish": True, "desc": "Byczy sygnał odwrócenia trendu spadkowego."},
            "CDLHARAMI": {"name": "Harami", "bullish": None, "desc": "Wskazuje na niezdecydowanie. Sprawdź, czy jest bycza czy niedźwiedzia."},
            "CDL3WHITESOLDIERS": {"name": "Trzej Biali Żołnierze", "bullish": True, "desc": "Silny byczy sygnał kontynuacji."},
            "CDL3BLACKCROWS": {"name": "Trzy Czarne Wrony", "bullish": False, "desc": "Silny niedźwiedzi sygnał kontynuacji."},
            "CDLEVENINGSTAR": {"name": "Gwiazda Wieczorna", "bullish": False, "desc": "Silny sygnał odwrócenia trendu wzrostowego."},
            "CDLDARKCLOUDCOVER": {"name": "Zasłona Ciemnej Chmury", "bullish": False, "desc": "Niedźwiedzi sygnał odwrócenia trendu wzrostowego."},
            "CDLDRAGONFLYDOJI": {"name": "Doji Ważka", "bullish": True, "desc": "Potencjalne odwrócenie trendu spadkowego."},
            "CDLGRAVESTONEDOJI": {"name": "Doji Nagrobek", "bullish": False, "desc": "Potencjalne odwrócenie trendu wzrostowego."},
            "CDLSHOOTINGSTAR": {"name": "Spadająca Gwiazda", "bullish": False, "desc": "Niedźwiedzi sygnał odwrócenia trendu wzrostowego."},
            "CDLHANGINGMAN": {"name": "Wisielec", "bullish": False, "desc": "Niedźwiedzi sygnał odwrócenia trendu wzrostowego."}
        }

        for pattern_func_name, pattern_info in talib_patterns.items():
            pattern_func = getattr(talib, pattern_func_name, None)
            if pattern_func:
                result = pattern_func(open_prices, high_prices, low_prices, close_prices)
                # Sprawdzamy ostatnią wartość, która jest najbardziej aktualna
                if result[-1] != 0:
                    strength = abs(result[-1])
                    is_bullish = result[-1] > 0

                    # Specjalne traktowanie dla Engulfing i Harami, gdzie kierunek zależy od wartości
                    if pattern_func_name == "CDLENGULFING":
                        pattern_info["bullish"] = is_bullish
                        pattern_info["desc"] = "Bycze objęcie" if is_bullish else "Niedźwiedzie objęcie"
                    elif pattern_func_name == "CDLHARAMI":
                        pattern_info["bullish"] = is_bullish
                        pattern_info["desc"] = "Bycze Harami" if is_bullish else "Niedźwiedzie Harami"

                    patterns.append(CandlestickPattern(
                        name=pattern_info["name"],
                        strength=int(strength),
                        bullish=pattern_info["bullish"] if pattern_info["bullish"] is not None else is_bullish,
                        description=pattern_info["desc"],
                        reliability=min(1.0, strength / 200.0) # Normalizacja siły do 0-1
                    ))
        return patterns

    def interpret_technical_signals(self, df: pd.DataFrame, current_price: float) -> List[TechnicalSignal]:
        """Interpretacja sygnałów z wskaźników technicznych"""
        signals = []
        if df.empty or len(df) < 30:
            return signals

        last_row = df.iloc[-1]
        prev_row = df.iloc[-2] if len(df) > 1 else None

        # RSI
        if 'rsi' in last_row and not pd.isna(last_row['rsi']):
            if last_row['rsi'] > 70:
                signals.append(TechnicalSignal("RSI", "Overbought", (last_row['rsi'] - 70) / 30, "Current", "RSI wskazuje na wykupienie."))
            elif last_row['rsi'] < 30:
                signals.append(TechnicalSignal("RSI", "Oversold", (30 - last_row['rsi']) / 30, "Current", "RSI wskazuje na wyprzedanie."))

        # MACD
        if all(col in last_row and not pd.isna(last_row[col]) for col in ['macd', 'macd_signal']):
            if last_row['macd'] > last_row['macd_signal'] and (prev_row is None or prev_row['macd'] <= prev_row['macd_signal']):
                signals.append(TechnicalSignal("MACD", "Bullish Crossover", 1.0, "Current", "MACD przecina linię sygnału w górę."))
            elif last_row['macd'] < last_row['macd_signal'] and (prev_row is None or prev_row['macd'] >= prev_row['macd_signal']):
                signals.append(TechnicalSignal("MACD", "Bearish Crossover", 1.0, "Current", "MACD przecina linię sygnału w dół."))

        # Bollinger Bands
        if all(col in last_row and not pd.isna(last_row[col]) for col in ['upper_bb', 'lower_bb', 'middle_bb']):
            if current_price > last_row['upper_bb']:
                signals.append(TechnicalSignal("Bollinger Bands", "Breakout Up", 0.8, "Current", "Cena powyżej górnej wstęgi Bollingera."))
            elif current_price < last_row['lower_bb']:
                signals.append(TechnicalSignal("Bollinger Bands", "Breakout Down", 0.8, "Current", "Cena poniżej dolnej wstęgi Bollingera."))
            elif last_row['upper_bb'] - last_row['lower_bb'] < (last_row['middle_bb'] * 0.01): # Wąskie wstęgi
                signals.append(TechnicalSignal("Bollinger Bands", "Squeeze", 0.6, "Current", "Wstęgi Bollingera zwężają się, potencjalny ruch wkrótce."))

        # ATR (volatility)
        if 'atr' in last_row and not pd.isna(last_row['atr']):
            if last_row['atr'] > df['atr'].mean() * 1.5:
                signals.append(TechnicalSignal("ATR", "High Volatility", 0.7, "Current", "Wysoka zmienność rynkowa."))
            elif last_row['atr'] < df['atr'].mean() * 0.5:
                signals.append(TechnicalSignal("ATR", "Low Volatility", 0.7, "Current", "Niska zmienność rynkowa."))

        # ADX
        if TALIB_AVAILABLE and 'adx' in df.columns and not pd.isna(last_row['adx']):
            if last_row['adx'] > 25:
                signals.append(TechnicalSignal("ADX", "Strong Trend", (last_row['adx'] - 25) / 75, "Current", "Wskazuje na silny trend."))
            elif last_row['adx'] < 20:
                signals.append(TechnicalSignal("ADX", "Weak Trend", (20 - last_row['adx']) / 20, "Current", "Wskazuje na słaby trend lub konsolidację."))

        # CCI
        if TALIB_AVAILABLE and 'cci' in df.columns and not pd.isna(last_row['cci']):
            if last_row['cci'] > 100:
                signals.append(TechnicalSignal("CCI", "Overbought", (last_row['cci'] - 100) / 200, "Current", "CCI wskazuje na wykupienie."))
            elif last_row['cci'] < -100:
                signals.append(TechnicalSignal("CCI", "Oversold", (-100 - last_row['cci']) / 200, "Current", "CCI wskazuje na wyprzedanie."))

        # Stochastic
        if TALIB_AVAILABLE and all(col in df.columns and not pd.isna(last_row[col]) for col in ['stoch_k', 'stoch_d']):
            if last_row['stoch_k'] > 80 and last_row['stoch_d'] > 80:
                signals.append(TechnicalSignal("Stochastic", "Overbought", (last_row['stoch_k'] - 80) / 20, "Current", "Stochastic wskazuje na wykupienie."))
            elif last_row['stoch_k'] < 20 and last_row['stoch_d'] < 20:
                signals.append(TechnicalSignal("Stochastic", "Oversold", (20 - last_row['stoch_k']) / 20, "Current", "Stochastic wskazuje na wyprzedanie."))
            if prev_row is not None and last_row['stoch_k'] > last_row['stoch_d'] and prev_row['stoch_k'] <= prev_row['stoch_d']:
                signals.append(TechnicalSignal("Stochastic", "Bullish Crossover", 0.9, "Current", "Stochastic K przecina D w górę."))
            elif prev_row is not None and last_row['stoch_k'] < last_row['stoch_d'] and prev_row['stoch_k'] >= prev_row['stoch_d']:
                signals.append(TechnicalSignal("Stochastic", "Bearish Crossover", 0.9, "Current", "Stochastic K przecina D w dół."))

        return signals

    def analyze_volume_profile(self, df: pd.DataFrame) -> Dict[str, float]:
        """Analiza profilu wolumenu (uproszczona)"""
        if df.empty:
            return {}

        # Obliczanie punktu kontroli (POC) i obszaru wartości (VA)
        # Uproszczona wersja: POC to cena z najwyższym wolumenem
        # VA to zakres cen, w którym handlowano X% całkowitego wolumenu

        price_bins = np.linspace(df['low'].min(), df['high'].max(), 50)
        volume_at_price = pd.cut(df['close'], bins=price_bins, labels=price_bins[:-1]).value_counts()

        if volume_at_price.empty:
            return {}

        poc_price = volume_at_price.idxmax()
        total_volume = df['volume'].sum()

        # Obliczanie Value Area (VA) - 70% wolumenu
        sorted_volumes = volume_at_price.sort_index()
        cumulative_volume = sorted_volumes.cumsum()

        va_low_idx = (cumulative_volume < (total_volume * 0.15)).sum()
        va_high_idx = (cumulative_volume < (total_volume * 0.85)).sum()

        va_low = sorted_volumes.index[va_low_idx] if va_low_idx < len(sorted_volumes) else df['low'].min()
        va_high = sorted_volumes.index[va_high_idx] if va_high_idx < len(sorted_volumes) else df['high'].max()

        return {
            "poc_price": float(poc_price),
            "value_area_low": float(va_low),
            "value_area_high": float(va_high),
            "total_volume": float(total_volume)
        }

    def identify_key_levels(self, df: pd.DataFrame) -> List[float]:
        """Identyfikacja kluczowych poziomów wsparcia i oporu (uproszczona)"""
        if df.empty:
            return []

        levels = []
        # Proste podejście: szczyty i dołki
        highs = df['high'].nlargest(5).values
        lows = df['low'].nsmallest(5).values

        # Dodaj poziomy Fibonacciego
        if len(df) > 1:
            max_price = df['high'].max()
            min_price = df['low'].min()
            price_range = max_price - min_price
            fib_levels = [0, 0.236, 0.382, 0.5, 0.618, 0.786, 1]
            for level in fib_levels:
                levels.append(min_price + price_range * level)

        all_levels = np.concatenate((highs, lows, levels))
        # Usuń duplikaty i posortuj
        unique_levels = np.unique(all_levels)
        # Zaokrągl do 2 miejsc po przecinku dla czytelności
        return sorted([round(float(l), 2) for l in unique_levels if not np.isnan(l)])

    def assess_market_sentiment(self, df: pd.DataFrame) -> MarketSentiment:
        """Ocena sentymentu rynkowego na podstawie RSI i wolumenu"""
        if df.empty or len(df) < 2: # Potrzeba co najmniej 2 wierszy dla price_change
            return MarketSentiment.NEUTRAL

        last_rsi = df['rsi'].iloc[-1] if 'rsi' in df.columns and not pd.isna(df['rsi'].iloc[-1]) else 50
        last_volume = df['volume'].iloc[-1] if 'volume' in df.columns and not pd.isna(df['volume'].iloc[-1]) else 1
        avg_volume = df['volume'].mean() if 'volume' in df.columns and not df['volume'].empty else 1

        sentiment_score = 0

        # Wpływ RSI
        if last_rsi > 70:
            sentiment_score += (last_rsi - 70) / 30 * 2 # Silny byczy
        elif last_rsi < 30:
            sentiment_score -= (30 - last_rsi) / 30 * 2 # Silny niedźwiedzi

        # Wpływ wolumenu (wzrost wolumenu przy wzroście ceny = byczy, przy spadku = niedźwiedzi)
        if last_volume > avg_volume * 1.5: # Znaczący wzrost wolumenu
            if df['close'].iloc[-1] > df['close'].iloc[-2]: # Cena rośnie
                sentiment_score += 1
            else: # Cena spada
                sentiment_score -= 1
        elif last_volume < avg_volume * 0.5: # Znaczący spadek wolumenu
            sentiment_score += 0.5 # Niska aktywność może wskazywać na niezdecydowanie

        # Wpływ zmian ceny
        price_change = (df['close'].iloc[-1] - df['open'].iloc[-1]) / df['open'].iloc[-1]
        if price_change > 0.01: # Znaczący wzrost
            sentiment_score += 1
        elif price_change < -0.01: # Znaczący spadek
            sentiment_score -= 1

        if sentiment_score > 2:
            return MarketSentiment.EXTREME_GREED
        elif sentiment_score > 1:
            return MarketSentiment.GREED
        elif sentiment_score < -2:
            return MarketSentiment.EXTREME_FEAR
        elif sentiment_score < -1:
            return MarketSentiment.FEAR
        else:
            return MarketSentiment.NEUTRAL

    def predict_price_movement(self, analysis: MarketAnalysis, df: pd.DataFrame, timeframe: str) -> PredictionModel:
        """Predykcja ruchu ceny na podstawie analizy"""
        if df.empty:
            return PredictionModel(timeframe, analysis.current_price, 0.0, 0.5, 0.5, [], 0.0, 0.0, 0.0, 0.0)

        # Wagi dla różnych czynników
        weights = self.config['prediction']

        # Obliczanie czynników wpływających na predykcję
        trend_score = 0
        if analysis.trend == TrendDirection.STRONG_BULLISH: trend_score = 2
        elif analysis.trend == TrendDirection.BULLISH: trend_score = 1
        elif analysis.trend == TrendDirection.BEARISH: trend_score = -1
        elif analysis.trend == TrendDirection.STRONG_BEARISH: trend_score = -2

        signal_score = sum([s.strength for s in analysis.technical_signals if 'Bullish' in s.signal]) - \
                       sum([s.strength for s in analysis.technical_signals if 'Bearish' in s.signal])

        pattern_score = sum([p.reliability for p in analysis.candlestick_patterns if p.bullish]) - \
                        sum([p.reliability for p in analysis.candlestick_patterns if not p.bullish])

        sentiment_score = analysis.market_sentiment_score / 5.0 # Normalizacja do -1 do 1

        # Agregacja wyników
        total_score = (trend_score * weights['weight_technical'] +
                       signal_score * weights['weight_technical'] +
                       pattern_score * weights['weight_pattern'] +
                       sentiment_score * weights['weight_sentiment'])

        # Predykcja ceny
        last_close = df['close'].iloc[-1]
        expected_change_percent = total_score * analysis.volatility * 0.5 # Zmiana procentowa zależna od zmienności
        prediction_price = last_close * (1 + expected_change_percent)

        # Konfidence i prawdopodobieństwa
        confidence = min(1.0, abs(total_score) / 5.0 + analysis.confidence * 0.5) # Łączenie siły sygnałów i ogólnej pewności

        probability_up = 0.5 + (total_score / 10.0)
        probability_down = 1 - probability_up

        probability_up = np.clip(probability_up, 0.1, 0.9)
        probability_down = np.clip(probability_down, 0.1, 0.9)

        # Kluczowe poziomy (np. najbliższe wsparcie/opór)
        key_levels = analysis.key_levels

        # Oczekiwana zmienność
        expected_volatility = analysis.volatility * (1 + abs(total_score) * 0.1) # Wzrost zmienności przy silnych sygnałach

        # Predykcja wolumenu
        volume_prediction = df['volume'].iloc[-1] * (1 + total_score * 0.1) # Wolumen rośnie z siłą sygnału

        # Risk-Reward Ratio (uproszczone)
        # Zakładamy, że stop loss jest na najbliższym poziomie wsparcia, a take profit na oporze
        risk_reward_ratio = 1.5 # Domyślna wartość
        if len(analysis.support_resistance_levels.get('support', [])) > 0 and \
           len(analysis.support_resistance_levels.get('resistance', [])) > 0:

            nearest_support = min(analysis.support_resistance_levels['support'], key=lambda x: abs(x - last_close))
            nearest_resistance = min(analysis.support_resistance_levels['resistance'], key=lambda x: abs(x - last_close))

            if total_score > 0 and nearest_resistance > last_close: # Byczy sygnał
                risk = abs(last_close - nearest_support)
                reward = abs(nearest_resistance - last_close)
                if risk > 0: risk_reward_ratio = reward / risk
            elif total_score < 0 and nearest_support < last_close: # Niedźwiedzi sygnał
                risk = abs(last_close - nearest_resistance)
                reward = abs(nearest_support - last_close)
                if risk > 0: risk_reward_ratio = reward / risk

        return PredictionModel(
            timeframe=timeframe,
            prediction_price=prediction_price,
            confidence=confidence,
            probability_up=probability_up,
            probability_down=probability_down,
            key_levels=key_levels,
            expected_volatility=expected_volatility,
            trend_strength=abs(total_score),
            volume_prediction=volume_prediction,
            risk_reward_ratio=risk_reward_ratio
        )

    def perform_market_analysis(self, instrument: WatchlistInstrument, df: pd.DataFrame, timeframes: List[str]) -> MarketAnalysis:
        """Przeprowadzanie kompleksowej analizy rynku dla danego instrumentu"""
        if df.empty:
            logger.warning(f"Brak danych dla {instrument.symbol}, pomijam analizę.")
            return MarketAnalysis(
                symbol=instrument.symbol,
                current_price=instrument.current_price,
                price_change_24h=0.0,
                volume_change_24h=0.0,
                trend=TrendDirection.NEUTRAL,
                signal_strength=SignalStrength.WEAK,
                market_sentiment=MarketSentiment.NEUTRAL,
                confidence=0.0,
                volatility=0.0,
                volume_analysis="Brak danych",
                key_levels=[],
                price_targets=[],
                risk_assessment="Brak danych",
                market_structure="Brak danych",
                candlestick_patterns=[],
                technical_signals=[],
                divergences=[],
                predictions={},
                fibonacci_levels={},
                elliott_wave_count="N/A",
                market_sentiment_score=0.0,
                fear_greed_impact="N/A",
                liquidity_analysis="N/A",
                order_flow_prediction="N/A",
                support_resistance_levels={},
                trend_analysis={},
                momentum_indicators={},
                volume_profile={},
                time_based_analysis={}
            )

        # Obliczanie wskaźników
        df_indicators = self.calculate_advanced_indicators(df.copy())

        current_price = instrument.current_price

        # Zmiany 24h
        price_change_24h = (current_price - df['close'].iloc[0]) / df['close'].iloc[0] if len(df) > 0 else 0.0
        volume_change_24h = (df['volume'].iloc[-1] - df['volume'].mean()) / df['volume'].mean() if len(df) > 0 else 0.0

        # Trendy i siła sygnału
        trend = self._determine_trend(df_indicators)
        signal_strength = self._determine_signal_strength(df_indicators)

        # Sentyment rynkowy
        market_sentiment = self.assess_market_sentiment(df_indicators)
        market_sentiment_score = self._calculate_sentiment_score(market_sentiment)

        # Zmienność
        volatility = df_indicators['atr'].iloc[-1] if 'atr' in df_indicators.columns and not pd.isna(df_indicators['atr'].iloc[-1]) else 0.01

        # Analiza wolumenu
        volume_analysis_str = self._analyze_volume(df_indicators)

        # Kluczowe poziomy
        key_levels = self.identify_key_levels(df_indicators)

        # Wzorce świecowe
        candlestick_patterns = self.detect_candlestick_patterns(df_indicators)

        # Sygnały techniczne
        technical_signals = self.interpret_technical_signals(df_indicators, current_price)

        # Predykcje dla różnych interwałów
        predictions = {}
        for tf in timeframes:
            # Użyjemy tych samych danych, ale w bardziej zaawansowanym systemie można by pobierać dane dla każdego TF
            predictions[tf] = self.predict_price_movement(
                MarketAnalysis( # Tworzymy tymczasową analizę dla predykcji
                    symbol=instrument.symbol,
                    current_price=current_price,
                    price_change_24h=price_change_24h,
                    volume_change_24h=volume_change_24h,
                    trend=trend,
                    signal_strength=signal_strength,
                    market_sentiment=market_sentiment,
                    confidence=0.0, # Będzie zaktualizowane w predict_price_movement
                    volatility=volatility,
                    volume_analysis=volume_analysis_str,
                    key_levels=key_levels,
                    price_targets=[],
                    risk_assessment="",
                    market_structure="",
                    candlestick_patterns=candlestick_patterns,
                    technical_signals=technical_signals,
                    divergences=[],
                    predictions={},
                    fibonacci_levels={},
                    elliott_wave_count="",
                    market_sentiment_score=market_sentiment_score,
                    fear_greed_impact="",
                    liquidity_analysis="",
                    order_flow_prediction="",
                    support_resistance_levels=self._identify_support_resistance(df_indicators),
                    trend_analysis={},
                    momentum_indicators={},
                    volume_profile={}
                ),
                df_indicators,
                tf
            )

        # Poziomy Fibonacciego (z funkcji identify_key_levels)
        fibonacci_levels = {f"fib_{i}": level for i, level in enumerate(key_levels) if i < 7} # Uproszczone

        # Uproszczone cele cenowe
        price_targets = self._generate_price_targets(current_price, key_levels, predictions.get('4h'))

        # Ocena ryzyka
        risk_assessment = self._assess_risk(volatility, signal_strength, market_sentiment)

        # Struktura rynku (uproszczona)
        market_structure = self._analyze_market_structure(df_indicators)

        # Dywergencje (placeholder)
        divergences = ["Brak wykrytych dywergencji"]

        # Analiza płynności (placeholder)
        liquidity_analysis = "Analiza płynności niedostępna w tej wersji."

        # Predykcja przepływu zleceń (placeholder)
        order_flow_prediction = "Predykcja przepływu zleceń niedostępna w tej wersji."

        # Poziomy wsparcia i oporu (bardziej szczegółowe)
        support_resistance_levels = self._identify_support_resistance(df_indicators)

        # Analiza trendu (szczegółowa)
        trend_analysis = self._detailed_trend_analysis(df_indicators)

        # Wskaźniki momentum
        momentum_indicators = self._get_momentum_indicators(df_indicators)

        # Profil wolumenu
        volume_profile = self.analyze_volume_profile(df_indicators)

        # Analiza czasowa
        time_based_analysis = self._perform_time_based_analysis(df_indicators)

        return MarketAnalysis(
            symbol=instrument.symbol,
            current_price=current_price,
            price_change_24h=price_change_24h,
            volume_change_24h=volume_change_24h,
            trend=trend,
            signal_strength=signal_strength,
            market_sentiment=market_sentiment,
            confidence=predictions.get('4h', PredictionModel('4h',0,0,0,0,[],0,0,0,0)).confidence, # Użyj pewności z predykcji 4h
            volatility=volatility,
            volume_analysis=volume_analysis_str,
            key_levels=key_levels,
            price_targets=price_targets,
            risk_assessment=risk_assessment,
            market_structure=market_structure,
            candlestick_patterns=candlestick_patterns,
            technical_signals=technical_signals,
            divergences=divergences,
            predictions=predictions,
            fibonacci_levels=fibonacci_levels,
            elliott_wave_count="N/A", # Placeholder
            market_sentiment_score=market_sentiment_score,
            fear_greed_impact="N/A", # Placeholder
            liquidity_analysis=liquidity_analysis,
            order_flow_prediction=order_flow_prediction,
            support_resistance_levels=support_resistance_levels,
            trend_analysis=trend_analysis,
            momentum_indicators=momentum_indicators,
            volume_profile=volume_profile,
            time_based_analysis=time_based_analysis
        )

    def _determine_trend(self, df: pd.DataFrame) -> TrendDirection:
        """Określanie kierunku i siły trendu na podstawie EMA i MACD"""
        if df.empty or len(df) < 50: # Potrzeba więcej danych dla wiarygodnego trendu
            return TrendDirection.NEUTRAL

        # Użyjemy 50-okresowej EMA dla trendu długoterminowego i 20-okresowej dla krótkoterminowego
        ema_long = SimpleIndicators.ema(df['close'], 50).iloc[-1] if not TALIB_AVAILABLE else talib.EMA(df['close'], 50).iloc[-1]
        ema_short = SimpleIndicators.ema(df['close'], 20).iloc[-1] if not TALIB_AVAILABLE else talib.EMA(df['close'], 20).iloc[-1]

        current_price = df['close'].iloc[-1]

        trend_score = 0
        if current_price > ema_long:
            trend_score += 1
        elif current_price < ema_long:
            trend_score -= 1

        if ema_short > ema_long:
            trend_score += 1
        elif ema_short < ema_long:
            trend_score -= 1

        macd_hist = df['macd_hist'].iloc[-1] if 'macd_hist' in df.columns and not pd.isna(df['macd_hist'].iloc[-1]) else 0
        if macd_hist > 0:
            trend_score += 0.5
        elif macd_hist < 0:
            trend_score -= 0.5

        if trend_score >= 2.5:
            return TrendDirection.STRONG_BULLISH
        elif trend_score >= 1:
            return TrendDirection.BULLISH
        elif trend_score <= -2.5:
            return TrendDirection.STRONG_BEARISH
        elif trend_score <= -1:
            return TrendDirection.BEARISH
        else:
            return TrendDirection.NEUTRAL

    def _determine_signal_strength(self, df: pd.DataFrame) -> SignalStrength:
        """Określanie siły sygnału na podstawie wskaźników"""
        if df.empty or len(df) < 2:
            return SignalStrength.WEAK

        strength_score = 0
        last_row = df.iloc[-1]

        # RSI
        if 'rsi' in last_row and not pd.isna(last_row['rsi']):
            if last_row['rsi'] > 70 or last_row['rsi'] < 30:
                strength_score += 1

        # MACD
        if all(col in last_row and not pd.isna(last_row[col]) for col in ['macd', 'macd_signal']):
            if (last_row['macd'] > last_row['macd_signal'] and df['macd'].iloc[-2] <= df['macd_signal'].iloc[-2]) or \
               (last_row['macd'] < last_row['macd_signal'] and df['macd'].iloc[-2] >= df['macd_signal'].iloc[-2]):
                strength_score += 1.5

        # Bollinger Bands
        if all(col in last_row and not pd.isna(last_row[col]) for col in ['upper_bb', 'lower_bb', 'middle_bb']):
            if last_row['close'] > last_row['upper_bb'] or last_row['close'] < last_row['lower_bb']:
                strength_score += 1.2
            elif (last_row['upper_bb'] - last_row['lower_bb']) < (last_row['middle_bb'] * 0.01): # Wąskie wstęgi
                strength_score += 0.8 # Squeeze

        # ATR
        if 'atr' in last_row and not pd.isna(last_row['atr']):
            if last_row['atr'] > df['atr'].mean() * 1.5:
                strength_score += 0.7 # Wysoka zmienność

        if strength_score >= 3:
            return SignalStrength.VERY_STRONG
        elif strength_score >= 2:
            return SignalStrength.STRONG
        elif strength_score >= 1:
            return SignalStrength.MODERATE
        else:
            return SignalStrength.WEAK

    def _analyze_volume(self, df: pd.DataFrame) -> str:
        """Analiza wolumenu"""
        if df.empty or len(df) < self.config['technical']['volume_ma_period'] + 1:
            return "Brak wystarczających danych do analizy wolumenu."

        last_volume = df['volume'].iloc[-1]
        volume_ma = df['volume_ma'].iloc[-1]

        if last_volume > volume_ma * self.config['alerts']['volume_spike_threshold']:
            return f"Wolumen znacznie powyżej średniej ({last_volume:.2f} vs {volume_ma:.2f}), co wskazuje na dużą aktywność."
        elif last_volume < volume_ma * 0.5:
            return f"Wolumen znacznie poniżej średniej ({last_volume:.2f} vs {volume_ma:.2f}), co wskazuje na niską aktywność."
        else:
            return f"Wolumen jest zgodny ze średnią ({last_volume:.2f} vs {volume_ma:.2f})."

    def _calculate_sentiment_score(self, sentiment: MarketSentiment) -> float:
        """Mapowanie sentymentu na wartość numeryczną"""
        if sentiment == MarketSentiment.EXTREME_GREED: return 5.0
        elif sentiment == MarketSentiment.GREED: return 3.0
        elif sentiment == MarketSentiment.NEUTRAL: return 0.0
        elif sentiment == MarketSentiment.FEAR: return -3.0
        elif sentiment == MarketSentiment.EXTREME_FEAR: return -5.0
        return 0.0

    def _generate_price_targets(self, current_price: float, key_levels: List[float], prediction: Optional[PredictionModel]) -> List[PriceTarget]:
        """Generowanie celów cenowych na podstawie kluczowych poziomów i predykcji"""
        targets = []

        # Cel z predykcji
        if prediction and prediction.confidence > self.config['analysis']['confidence_threshold']:
            targets.append(PriceTarget(
                level=prediction.prediction_price,
                probability=prediction.probability_up if prediction.prediction_price > current_price else prediction.probability_down,
                timeframe=prediction.timeframe,
                resistance_support="Predykcja",
                description=f"Przewidywana cena w ciągu {prediction.timeframe} z pewnością {prediction.confidence:.2f}."
            ))

        # Cele z kluczowych poziomów
        sorted_levels = sorted(key_levels)
        for level in sorted_levels:
            if abs(level - current_price) / current_price < 0.005: # Zbyt blisko obecnej ceny
                continue

            if level > current_price: # Potencjalny opór / cel wzrostowy
                targets.append(PriceTarget(
                    level=level,
                    probability=0.6, # Domyślne prawdopodobieństwo
                    timeframe="Różne",
                    resistance_support="Opór",
                    description=f"Kluczowy poziom oporu."
                ))
            else: # Potencjalne wsparcie / cel spadkowy
                targets.append(PriceTarget(
                    level=level,
                    probability=0.6,
                    timeframe="Różne",
                    resistance_support="Wsparcie",
                    description=f"Kluczowy poziom wsparcia."
                ))

        # Sortuj cele według odległości od bieżącej ceny
        targets.sort(key=lambda x: abs(x.level - current_price))
        return targets[:5] # Zwróć top 5

    def _assess_risk(self, volatility: float, signal_strength: SignalStrength, market_sentiment: MarketSentiment) -> str:
        """Ocena ryzyka na podstawie zmienności, siły sygnału i sentymentu"""
        risk_score = 0

        if volatility > self.config['alerts']['volatility_threshold']:
            risk_score += 2 # Wysoka zmienność zwiększa ryzyko
        elif volatility < self.config['alerts']['volatility_threshold'] / 2:
            risk_score -= 1 # Niska zmienność może zmniejszać ryzyko (ale też szanse)

        if signal_strength in [SignalStrength.WEAK, SignalStrength.VERY_WEAK]:
            risk_score += 1.5 # Słabe sygnały zwiększają ryzyko
        elif signal_strength in [SignalStrength.STRONG, SignalStrength.VERY_STRONG]:
            risk_score -= 1 # Silne sygnały zmniejszają ryzyko

        if market_sentiment in [MarketSentiment.EXTREME_FEAR, MarketSentiment.EXTREME_GREED]:
            risk_score += 1 # Ekstremalny sentyment zwiększa ryzyko

        if risk_score >= 3:
            return "Wysokie ryzyko 🔴"
        elif risk_score >= 1.5:
            return "Umiarkowane ryzyko 🟠"
        else:
            return "Niskie ryzyko 🟢"

    def _analyze_market_structure(self, df: pd.DataFrame) -> str:
        """Analiza struktury rynku (uproszczona)"""
        if df.empty or len(df) < 20:
            return "Brak wystarczających danych do analizy struktury rynku."

        # Szukanie wyższych szczytów/dołków lub niższych szczytów/dołków
        recent_highs = df['high'].iloc[-5:].max()
        recent_lows = df['low'].iloc[-5:].min()

        previous_highs = df['high'].iloc[-10:-5].max()
        previous_lows = df['low'].iloc[-10:-5].min()

        if recent_highs > previous_highs and recent_lows > previous_lows:
            return "Struktura bycza (wyższe szczyty i wyższe dołki)."
        elif recent_highs < previous_highs and recent_lows < previous_lows:
            return "Struktura niedźwiedzia (niższe szczyty i niższe dołki)."
        else:
            return "Struktura konsolidacji lub mieszana."

    def _identify_support_resistance(self, df: pd.DataFrame) -> Dict[str, List[float]]:
        """Identyfikacja poziomów wsparcia i oporu na podstawie historycznych danych (uproszczona)"""
        if df.empty:
            return {"support": [], "resistance": []}

        support_levels = []
        resistance_levels = []

        # Szczyty i dołki jako potencjalne S/R
        highs = df['high'].sort_values(ascending=False).drop_duplicates().head(5).tolist()
        lows = df['low'].sort_values().drop_duplicates().head(5).tolist()

        # Dodaj średnie kroczące jako dynamiczne S/R
        if 'middle_bb' in df.columns and not pd.isna(df['middle_bb'].iloc[-1]):
            support_levels.append(df['middle_bb'].iloc[-1])
            resistance_levels.append(df['middle_bb'].iloc[-1]) # Może działać jako oba

        # Dodaj kluczowe poziomy z analizy wolumenu
        volume_profile = self.analyze_volume_profile(df)
        if volume_profile:
            if volume_profile.get('value_area_low') is not None:
                support_levels.append(volume_profile.get('value_area_low'))
            if volume_profile.get('value_area_high') is not None:
                resistance_levels.append(volume_profile.get('value_area_high'))
            if volume_profile.get('poc_price') is not None:
                support_levels.append(volume_profile.get('poc_price'))
                resistance_levels.append(volume_profile.get('poc_price'))

        # Usuń None i zaokrągl
        support_levels = sorted(list(set([round(l, 2) for l in support_levels + lows if l is not None])))
        resistance_levels = sorted(list(set([round(l, 2) for l in resistance_levels + highs if l is not None])))

        return {"support": support_levels, "resistance": resistance_levels}

    def _detailed_trend_analysis(self, df: pd.DataFrame) -> Dict[str, Any]:
        """Szczegółowa analiza trendu"""
        if df.empty or len(df) < 100:
            return {"status": "Brak wystarczających danych."}

        trend_status = self._determine_trend(df)

        # Analiza kąta nachylenia średnich kroczących
        # Upewnij się, że jest wystarczająco danych do obliczenia nachylenia
        ema_short_slope = 0.0
        if len(df['close'].tail(20)) >= 2:
            ema_short_slope = np.polyfit(range(len(df['close'].tail(20))), df['close'].tail(20), 1)[0]

        ema_long_slope = 0.0
        if len(df['close'].tail(50)) >= 2:
            ema_long_slope = np.polyfit(range(len(df['close'].tail(50))), df['close'].tail(50), 1)[0]

        return {
            "status": trend_status.value,
            "short_term_slope": round(ema_short_slope, 4),
            "long_term_slope": round(ema_long_slope, 4),
            "trend_strength_score": self._calculate_trend_strength_score(df)
        }

    def _calculate_trend_strength_score(self, df: pd.DataFrame) -> float:
        """Obliczanie numerycznej siły trendu"""
        if df.empty or len(df) < 50:
            return 0.0

        adx = df['adx'].iloc[-1] if 'adx' in df.columns and not pd.isna(df['adx'].iloc[-1]) else 0

        # Normalizacja ADX do 0-1
        adx_score = min(1.0, adx / 50.0) # ADX > 50 to bardzo silny trend

        # Dodatkowo można uwzględnić odległość ceny od EMA
        ema_long = SimpleIndicators.ema(df['close'], 50).iloc[-1] if not TALIB_AVAILABLE else talib.EMA(df['close'], 50).iloc[-1]
        price_distance_from_ema = 0.0
        if ema_long != 0:
            price_distance_from_ema = abs(df['close'].iloc[-1] - ema_long) / ema_long

        # Im większa odległość, tym silniejszy trend (ale też większe ryzyko korekty)
        # Ograniczamy wpływ, żeby nie zdominował
        distance_score = min(1.0, price_distance_from_ema * 5)

        return (adx_score * 0.7 + distance_score * 0.3) # Prosta waga

    def _get_momentum_indicators(self, df: pd.DataFrame) -> Dict[str, float]:
        """Pobieranie wartości wskaźników momentum"""
        if df.empty:
            return {}

        momentum_data = {}
        last_row = df.iloc[-1]

        if 'rsi' in last_row and not pd.isna(last_row['rsi']):
            momentum_data['RSI'] = round(last_row['rsi'], 2)
        if 'macd' in last_row and not pd.isna(last_row['macd']):
            momentum_data['MACD'] = round(last_row['macd'], 4)
        if 'macd_hist' in last_row and not pd.isna(last_row['macd_hist']):
            momentum_data['MACD Hist'] = round(last_row['macd_hist'], 4)
        if 'stoch_k' in last_row and not pd.isna(last_row['stoch_k']):
            momentum_data['Stoch %K'] = round(last_row['stoch_k'], 2)
        if 'stoch_d' in last_row and not pd.isna(last_row['stoch_d']):
            momentum_data['Stoch %D'] = round(last_row['stoch_d'], 2)
        if 'cci' in last_row and not pd.isna(last_row['cci']):
            momentum_data['CCI'] = round(last_row['cci'], 2)

        return momentum_data

    def _perform_time_based_analysis(self, df: pd.DataFrame) -> Dict[str, Any]:
        """Analiza oparta na czasie (np. sesje, cykle) - uproszczona"""
        if df.empty:
            return {"status": "Brak danych."}

        # Prosta analiza: czy obecna godzina jest w "aktywnych" godzinach handlowych
        current_hour = datetime.now().hour
        is_active_session = 9 <= current_hour <= 17 # Przykład dla sesji europejskiej

        # Analiza zmienności w ciągu dnia (uproszczona)
        df['hour'] = df.index.hour
        if len(df['close']) > 1: # Ensure there's enough data for pct_change()
            hourly_volatility = df.groupby('hour')['close'].apply(lambda x: x.pct_change().std()).fillna(0)
        else:
            hourly_volatility = pd.Series() # Empty series if no data

        most_volatile_hour = hourly_volatility.idxmax() if not hourly_volatility.empty else "N/A"
        least_volatile_hour = hourly_volatility.idxmin() if not hourly_volatility.empty else "N/A"

        return {
            "is_active_session": is_active_session,
            "current_hour": current_hour,
            "most_volatile_hour": most_volatile_hour,
            "least_volatile_hour": least_volatile_hour
        }

    async def update_watchlist_data(self):
        """Aktualizacja danych dla wszystkich instrumentów w watchliście
           Wykorzystuje fetch_ticker i fetch_positions z ccxt.
        """
        with Progress(
            SpinnerColumn(),
            TextColumn("[progress.description]{task.description}"),
            transient=True
        ) as progress:
            task = progress.add_task("[cyan]Aktualizowanie danych rynkowych...", total=len(self.watchlist))

            for instrument in self.watchlist:
                try:
                    if not self.demo_mode and self.exchange:
                        # Aktualizacja tickera
                        ticker = await self.exchange.fetch_ticker(instrument.symbol)
                        instrument.current_price = float(ticker.get('last', instrument.current_price))
                        instrument.change_24h = float(ticker.get('percentage', 0.0))
                        instrument.volume_24h = float(ticker.get('quoteVolume', 0.0))

                        # Aktualizacja pozycji
                        # ccxt fetch_positions zwraca listę wszystkich pozycji, filtrujemy po symbolu
                        positions = await self.exchange.fetch_positions()
                        pos_data = next((p for p in positions if p['symbol'] == instrument.symbol), None)
                        if pos_data:
                            instrument.active_position = pos_data.get('contracts', 0) != 0
                            instrument.position_size = float(pos_data.get('contracts', 0.0))
                            instrument.entry_price = float(pos_data.get('entryPrice', 0.0))
                            instrument.unrealized_pnl = float(pos_data.get('unrealisedPnl', 0.0))
                        else: # Pozycja mogła zostać zamknięta lub nie istnieje
                            instrument.active_position = False
                            instrument.position_size = 0.0
                            instrument.entry_price = 0.0
                            instrument.unrealized_pnl = 0.0

                    elif self.demo_mode: # Tryb demo
                        # Symulacja zmian ceny i wolumenu
                        change_factor = np.random.uniform(-0.02, 0.02)
                        instrument.current_price *= (1 + change_factor)
                        instrument.change_24h = change_factor * 100
                        instrument.volume_24h = instrument.volume_24h * np.random.uniform(0.9, 1.1)

                        # Symulacja P&L w trybie demo
                        if instrument.active_position:
                            instrument.unrealized_pnl = (instrument.current_price - instrument.entry_price) * instrument.position_size
                            # Losowe zamknięcie pozycji w demo
                            if np.random.rand() < 0.01: # 1% szans na zamknięcie
                                instrument.active_position = False
                                instrument.position_size = 0.0
                                instrument.entry_price = 0.0
                                instrument.unrealized_pnl = 0.0
                                console.print(f"[yellow]Symulacja: Pozycja {instrument.symbol} została zamknięta.[/yellow]")

                    instrument.last_update = datetime.now()
                    progress.update(task, advance=1, description=f"[cyan]Aktualizowanie {instrument.symbol}...")
                except Exception as e:
                    logger.error(f"Błąd aktualizacji danych dla {instrument.symbol}: {e}")
                    console.print(f"[red]❌ Błąd aktualizacji {instrument.symbol}: {e}[/red]")
                    progress.update(task, advance=1, description=f"[red]Błąd {instrument.symbol}!")
            progress.console.print("[green]✅ Dane rynkowe zaktualizowane.[/green]")

    async def run_analysis(self):
        """Uruchamianie analizy dla wszystkich instrumentów"""
        timeframes_to_analyze = ['15m', '1h', '4h', '1d']

        with Progress(
            SpinnerColumn(),
            TextColumn("[progress.description]{task.description}"),
            transient=True
        ) as progress:
            task = progress.add_task("[cyan]Przeprowadzanie analizy...", total=len(self.watchlist))

            for instrument in self.watchlist:
                try:
                    df = await self.get_market_data(instrument.symbol, '1h')
                    if df.empty:
                        logger.warning(f"Brak danych historycznych dla {instrument.symbol}, pomijam szczegółową analizę.")
                        instrument.analysis = None # Ustawiamy na None, jeśli brak danych
                        progress.update(task, advance=1, description=f"[yellow]Pominięto {instrument.symbol} (brak danych).")
                        continue

                    analysis = self.perform_market_analysis(instrument, df, timeframes_to_analyze)
                    instrument.analysis = analysis
                    self.analysis_history.append(analysis)

                    # Aktualizacja statystyk predykcji (uproszczone)
                    for tf, pred in analysis.predictions.items():
                        self.prediction_stats['total_predictions'] += 1
                        # Symulacja poprawności predykcji
                        if np.random.rand() < pred.confidence: # Im większa pewność, tym większa szansa na poprawność
                            self.prediction_stats['correct_predictions'] += 1
                            self.prediction_stats['accuracy_by_timeframe'].setdefault(tf, {'correct': 0, 'total': 0})['correct'] += 1
                            self.prediction_stats['accuracy_by_instrument'].setdefault(instrument.symbol, {'correct': 0, 'total': 0})['correct'] += 1
                        self.prediction_stats['accuracy_by_timeframe'].setdefault(tf, {'correct': 0, 'total': 0})['total'] += 1
                        self.prediction_stats['accuracy_by_instrument'].setdefault(instrument.symbol, {'correct': 0, 'total': 0})['total'] += 1

                    progress.update(task, advance=1, description=f"[cyan]Analiza {instrument.symbol} zakończona.")
                except Exception as e:
                    logger.error(f"Błąd podczas analizy {instrument.symbol}: {e}")
                    console.print(f"[red]❌ Błąd analizy {instrument.symbol}: {e}[/red]")
                    progress.update(task, advance=1, description=f"[red]Błąd {instrument.symbol}!")
            progress.console.print("[green]✅ Analiza rynku zakończona.[/green]")

    def display_dashboard(self):
        """Wyświetlanie interaktywnego dashboardu w konsoli Rich"""
        layout = Layout(name="root")

        layout.split(
            Layout(name="header", size=3),
            Layout(name="main"),
            Layout(name="footer", size=5)
        )

        layout["main"].split_row(
            Layout(name="watchlist", minimum_size=50),
            Layout(name="details")
        )
        layout["details"].split_column(
            Layout(name="selected_instrument_details", minimum_size=20),
            Layout(name="prediction_summary", minimum_size=10),
            Layout(name="alerts_and_logs", minimum_size=10)
        )

        layout["header"].update(self._make_header())
        layout["footer"].update(self._make_footer())

        with Live(layout, screen=True, redirect_stderr=False, refresh_per_second=4) as live:
            selected_symbol = None
            # Początkowy wybór instrumentu, jeśli dostępny
            if self.watchlist:
                selected_symbol = self.watchlist[0].symbol

            while self.is_running:
                # Aktualizacja watchlisty
                watchlist_panel = self._make_watchlist_panel(selected_symbol)
                layout["watchlist"].update(watchlist_panel)

                # Aktualizacja szczegółów wybranego instrumentu
                selected_instrument = next((i for i in self.watchlist if i.symbol == selected_symbol), None)
                if selected_instrument and selected_instrument.analysis:
                    layout["selected_instrument_details"].update(self._make_selected_instrument_details_panel(selected_instrument))
                    layout["prediction_summary"].update(self._make_prediction_summary_panel(selected_instrument.analysis))
                else:
                    layout["selected_instrument_details"].update(Panel(Text("Wybierz instrument, aby zobaczyć szczegóły analizy.", justify="center"), title="[bold blue]Szczegóły Instrumentu[/bold blue]"))
                    layout["prediction_summary"].update(Panel(Text("Brak predykcji do wyświetlenia.", justify="center"), title="[bold blue]Podsumowanie Predykcji[/bold blue]"))

                # Aktualizacja alertów i logów (uproszczone)
                layout["alerts_and_logs"].update(self._make_alerts_and_logs_panel())

                live.refresh()
                time.sleep(self.config['analysis']['refresh_interval'])

                # Sprawdzenie, czy użytkownik chce zmienić wybrany instrument
                if not self.is_running: # Sprawdź ponownie, jeśli wątek został zatrzymany
                    break

                # Symulacja interakcji użytkownika (w prawdziwej aplikacji byłoby to input())
                # Zmieniono logikę, aby użytkownik mógł wybrać symbol, jeśli chce
                # Jeśli chcesz w pełni interaktywny wybór, musiałbyś użyć Rich.prompt.Prompt
                # Tutaj jest uproszczona symulacja, która losowo zmienia wybrany symbol
                if np.random.rand() < 0.1: # 10% szans na zmianę wybranego instrumentu
                    if self.watchlist:
                        new_selected_symbol = np.random.choice([i.symbol for i in self.watchlist])
                        if new_selected_symbol != selected_symbol:
                            console.print(f"[bold magenta]Zmieniono wybrany instrument na: {new_selected_symbol}[/bold magenta]")
                            selected_symbol = new_selected_symbol

    def _make_header(self) -> Panel:
        """Tworzy nagłówek dashboardu."""
        title = Text("KuCoin Futures Analyzer", justify="center", style="bold white on blue")
        subtitle = Text(f"Ostatnia aktualizacja: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}", justify="center", style="dim white")
        return Panel(Columns([title, subtitle]), style="blue", box=ROUNDED)

    def _make_footer(self) -> Panel:
        """Tworzy stopkę dashboardu."""
        total_predictions = self.prediction_stats['total_predictions']
        correct_predictions = self.prediction_stats['correct_predictions']
        accuracy = (correct_predictions / total_predictions * 100) if total_predictions > 0 else 0

        accuracy_text = Text(f"Dokładność predykcji: {accuracy:.2f}% ({correct_predictions}/{total_predictions})", style="bold green" if accuracy > 60 else "bold yellow" if accuracy > 40 else "bold red")

        status_text = Text("Status: Działa " + ("[yellow](Tryb Demo)" if self.demo_mode else "[green](Tryb Live)"), style="bold white")

        return Panel(Columns([status_text, Align(accuracy_text, align="right")]), style="blue", box=ROUNDED)

    def _make_watchlist_panel(self, selected_symbol: Optional[str]) -> Panel:
        """Tworzy panel watchlisty."""
        table = Table(
            title="[bold white]Watchlista Instrumentów Futures[/bold white]",
            show_header=True,
            header_style="bold magenta",
            border_style="blue",
            box=ROUNDED
        )
        table.add_column("Symbol", style="cyan", justify="left")
        table.add_column("Nazwa", style="white", justify="left")
        table.add_column("Cena", style="green", justify="right")
        table.add_column("24h Zmiana", style="white", justify="right")
        table.add_column("24h Wolumen", style="white", justify="right")
        table.add_column("Trend", style="white", justify="left")
        table.add_column("Sygnał", style="white", justify="left")
        table.add_column("Sentyment", style="white", justify="left")
        table.add_column("Pozycja", style="white", justify="left")
        table.add_column("P&L", style="white", justify="right")

        for instrument in self.watchlist:
            price_color = "green" if instrument.change_24h >= 0 else "red"
            trend_color = "green" if instrument.analysis and instrument.analysis.trend in [TrendDirection.BULLISH, TrendDirection.STRONG_BULLISH] else \
                          "red" if instrument.analysis and instrument.analysis.trend in [TrendDirection.BEARISH, TrendDirection.STRONG_BEARISH] else "yellow"
            sentiment_color = "green" if instrument.analysis and instrument.analysis.market_sentiment in [MarketSentiment.GREED, MarketSentiment.EXTREME_GREED] else \
                              "red" if instrument.analysis and instrument.analysis.market_sentiment in [MarketSentiment.FEAR, MarketSentiment.EXTREME_FEAR] else "yellow"

            position_status = "[bold green]Otwarta[/bold green]" if instrument.active_position else "[dim white]Brak[/dim white]"
            pnl_color = "green" if instrument.unrealized_pnl >= 0 else "red"

            row_style = "on #333333" if instrument.symbol == selected_symbol else ""

            table.add_row(
                Text(instrument.symbol, style=f"bold {row_style}"),
                Text(instrument.name, style=row_style),
                Text(f"{instrument.current_price:,.2f}", style=f"{price_color} {row_style}"),
                Text(f"{instrument.change_24h:+.2f}%", style=f"{price_color} {row_style}"),
                Text(f"{instrument.volume_24h:,.0f}", style=f"white {row_style}"),
                Text(instrument.analysis.trend.value if instrument.analysis else "N/A", style=f"{trend_color} {row_style}"),
                Text(instrument.analysis.signal_strength.value if instrument.analysis else "N/A", style=f"white {row_style}"),
                Text(instrument.analysis.market_sentiment.value if instrument.analysis else "N/A", style=f"{sentiment_color} {row_style}"),
                Text(position_status, style=row_style),
                Text(f"{instrument.unrealized_pnl:,.2f}", style=f"{pnl_color} {row_style}")
            )
        return Panel(table, title="[bold blue]Watchlista[/bold blue]", border_style="blue", box=ROUNDED)

    def _make_selected_instrument_details_panel(self, instrument: WatchlistInstrument) -> Panel:
        """Tworzy panel ze szczegółami wybranego instrumentu."""
        if not instrument.analysis:
            return Panel(Text("Brak danych analitycznych dla tego instrumentu.", justify="center"), title="[bold blue]Szczegóły Instrumentu[/bold blue]")

        analysis = instrument.analysis

        details_tree = Tree(f"[bold white]{instrument.name} ({instrument.symbol}) - Szczegółowa Analiza[/bold white]")

        details_tree.add(f"Cena: [green]{analysis.current_price:,.2f}[/green] (24h: {analysis.price_change_24h:+.2f}%)")
        details_tree.add(f"Wolumen 24h: [cyan]{analysis.volume_change_24h:+.2f}%[/cyan]")
        details_tree.add(f"Trend: [bold {self._get_trend_color(analysis.trend)}]{analysis.trend.value}[/bold {self._get_trend_color(analysis.trend)}]")
        details_tree.add(f"Siła Sygnału: [bold white]{analysis.signal_strength.value}[/bold white]")
        details_tree.add(f"Sentyment Rynkowy: [bold {self._get_sentiment_color(analysis.market_sentiment)}]{analysis.market_sentiment.value}[/bold {self._get_sentiment_color(analysis.market_sentiment)}]")
        details_tree.add(f"Pewność Analizy: [yellow]{analysis.confidence:.2f}[/yellow]")
        details_tree.add(f"Zmienność (ATR): [purple]{analysis.volatility:.4f}[/purple]")
        details_tree.add(f"Ocena Ryzyka: {analysis.risk_assessment}")
        details_tree.add(f"Struktura Rynku: [white]{analysis.market_structure}[/white]")
        details_tree.add(f"Analiza Wolumenu: [white]{analysis.volume_analysis}[/white]")

        # Pozycja
        position_info = details_tree.add("[bold yellow]Pozycja:[/bold yellow]")
        if instrument.active_position:
            position_info.add(f"Status: [bold green]Otwarta[/bold green]")
            position_info.add(f"Rozmiar: [white]{instrument.position_size:.4f}[/white]")
            position_info.add(f"Cena Wejścia: [white]{instrument.entry_price:,.2f}[/white]")
            position_info.add(f"Niezrealizowany P&L: [bold {self._get_pnl_color(instrument.unrealized_pnl)}]{instrument.unrealized_pnl:,.2f}[/bold {self._get_pnl_color(instrument.unrealized_pnl)}]")
        else:
            position_info.add("[dim white]Brak aktywnej pozycji.[/dim white]")

        # Kluczowe poziomy
        key_levels_node = details_tree.add("[bold yellow]Kluczowe Poziomy:[/bold yellow]")
        for level in analysis.key_levels:
            key_levels_node.add(f"• {level:,.2f}")

        # Wzorce świecowe
        if self.config['display']['show_patterns'] and analysis.candlestick_patterns:
            patterns_node = details_tree.add("[bold yellow]Wzorce Świecowe:[/bold yellow]")
            for pattern in analysis.candlestick_patterns:
                color = "green" if pattern.bullish else "red"
                patterns_node.add(f"• [{color}]{pattern.name}[/{color}] (Siła: {pattern.strength}, Niezawodność: {pattern.reliability:.2f}) - {pattern.description}")
        elif self.config['display']['show_patterns']:
            details_tree.add("[dim]Brak wykrytych wzorców świecowych.[/dim]")

        # Sygnały techniczne
        if analysis.technical_signals:
            signals_node = details_tree.add("[bold yellow]Sygnały Techniczne:[/bold yellow]")
            for signal in analysis.technical_signals:
                signals_node.add(f"• [blue]{signal.indicator}[/blue]: [white]{signal.signal}[/white] (Siła: {signal.strength:.2f}) - {signal.description}")
        else:
            details_tree.add("[dim]Brak aktywnych sygnałów technicznych.[/dim]")

        # Dodatkowe analizy
        additional_analysis_node = details_tree.add("[bold yellow]Dodatkowe Analizy:[/bold yellow]")
        additional_analysis_node.add(f"Fibonacci: {json.dumps(analysis.fibonacci_levels)}")
        additional_analysis_node.add(f"Analiza Trendu: {json.dumps(analysis.trend_analysis)}")
        additional_analysis_node.add(f"Momentum: {json.dumps(analysis.momentum_indicators)}")
        additional_analysis_node.add(f"Profil Wolumenu: {json.dumps(analysis.volume_profile)}")
        additional_analysis_node.add(f"Analiza Czasowa: {json.dumps(analysis.time_based_analysis)}")
        additional_analysis_node.add(f"Wsparcie/Opór: {json.dumps(analysis.support_resistance_levels)}")

        return Panel(details_tree, title="[bold blue]Szczegóły Instrumentu[/bold blue]", border_style="blue", box=ROUNDED)

    def _make_prediction_summary_panel(self, analysis: MarketAnalysis) -> Panel:
        """Tworzy panel podsumowania predykcji."""
        if not self.config['display']['show_predictions'] or not analysis.predictions:
            return Panel(Text("Predykcje wyłączone lub brak danych.", justify="center"), title="[bold blue]Podsumowanie Predykcji[/bold blue]")

        prediction_table = Table(
            title="[bold white]Podsumowanie Predykcji Cenowych[/bold white]",
            show_header=True,
            header_style="bold green",
            border_style="green",
            box=ROUNDED
        )
        prediction_table.add_column("Interwał", style="cyan", justify="left")
        prediction_table.add_column("Przewidywana Cena", style="white", justify="right")
        prediction_table.add_column("Pewność", style="white", justify="right")
        prediction_table.add_column("Prawd. Wzrostu", style="white", justify="right")
        prediction_table.add_column("Prawd. Spadku", style="white", justify="right")
        prediction_table.add_column("Ryzyko/Zysk", style="white", justify="right")

        for tf, pred in analysis.predictions.items():
            if pred.confidence >= self.config['analysis']['confidence_threshold']:
                prediction_table.add_row(
                    tf,
                    f"{pred.prediction_price:,.2f}",
                    f"{pred.confidence:.2f}",
                    f"{pred.probability_up:.2%}",
                    f"{pred.probability_down:.2%}",
                    f"{pred.risk_reward_ratio:.2f}:1"
                )
        if not prediction_table.rows:
            return Panel(Text("Brak predykcji powyżej progu pewności.", justify="center"), title="[bold blue]Podsumowanie Predykcji[/bold blue]")

        return Panel(prediction_table, title="[bold blue]Podsumowanie Predykcji[/bold blue]", border_style="green", box=ROUNDED)

    def _make_alerts_and_logs_panel(self) -> Panel:
        """Tworzy panel alertów i logów (uproszczone)."""
        log_content = Text("")

        # Symulacja logów
        simulated_logs = [
            (datetime.now() - timedelta(seconds=5), "INFO", "System uruchomiony."),
            (datetime.now() - timedelta(seconds=4), "DEBUG", "Pobieram dane dla BTC/USDT."),
            (datetime.now() - timedelta(seconds=3), "WARNING", "Niska płynność na ETH/USDT."),
            (datetime.now() - timedelta(seconds=2), "INFO", "Analiza zakończona dla wszystkich instrumentów."),
            (datetime.now() - timedelta(seconds=1), "ERROR", "Błąd połączenia z API dla ADA/USDT.")
        ]

        for log_time, level, message in simulated_logs:
            log_content.append(f"{log_time.strftime('%H:%M:%S')} - {level}: {message}\n", style="dim")

        # Dodaj przykładowe alerty
        if np.random.rand() < 0.05: # 5% szans na alert
            alert_type = np.random.choice(["price_spike", "volume_spike", "pattern_detected"])
            if alert_type == "price_spike":
                log_content.append("\n[bold red]🚨 ALERT: Wykryto gwałtowną zmianę ceny na BTC/USDT![/bold red]\n")
            elif alert_type == "volume_spike":
                log_content.append("\n[bold yellow]⚠️ ALERT: Wykryto znaczny wzrost wolumenu na ETH/USDT![/bold yellow]\n")
            elif alert_type == "pattern_detected":
                log_content.append("\n[bold green]✨ ALERT: Wykryto byczy wzorzec świecowy na SOL/USDT![/bold green]\n")

        return Panel(log_content, title="[bold red]Alerty i Logi[/bold red]", border_style="red", box=ROUNDED)

    def _get_trend_color(self, trend: TrendDirection) -> str:
        """Zwraca kolor dla trendu."""
        if trend in [TrendDirection.STRONG_BULLISH, TrendDirection.BULLISH]:
            return "green"
        elif trend in [TrendDirection.STRONG_BEARISH, TrendDirection.BEARISH]:
            return "red"
        else:
            return "yellow"

    def _get_sentiment_color(self, sentiment: MarketSentiment) -> str:
        """Zwraca kolor dla sentymentu."""
        if sentiment in [MarketSentiment.GREED, MarketSentiment.EXTREME_GREED]:
            return "green"
        elif sentiment in [MarketSentiment.FEAR, MarketSentiment.EXTREME_FEAR]:
            return "red"
        else:
            return "yellow"

    def _get_pnl_color(self, pnl: float) -> str:
        """Zwraca kolor dla P&L."""
        if pnl >= 0:
            return "green"
        else:
            return "red"

    async def start(self):
        """Rozpoczyna działanie systemu analizy."""
        self.is_running = True
        console.print("[bold green]System analizy KuCoin Futures uruchomiony...[/bold green]")

        # Uruchomienie pętli aktualizacji danych i analizy w tle
        async def analysis_loop():
            while self.is_running:
                await self.update_watchlist_data()
                await self.run_analysis()
                await asyncio.sleep(self.config['analysis']['refresh_interval'])

        # Uruchomienie pętli analizy w osobnym zadaniu asyncio
        analysis_task = asyncio.create_task(analysis_loop())

        # Uruchomienie dashboardu w głównym wątku
        self.display_dashboard()

        await analysis_task # Poczekaj na zakończenie zadania analizy
        console.print("[bold red]System analizy KuCoin Futures zatrzymany.[/bold red]")

    def stop(self):
        """Zatrzymuje działanie systemu analizy."""
        self.is_running = False
        console.print("[bold yellow]Zatrzymywanie systemu analizy...[/bold yellow]")
        if self.exchange:
            asyncio.run(self.exchange.close()) # Zamknij połączenie ccxt


if __name__ == "__main__":
    analyzer = AdvancedKuCoinAnalyzer()
    try:
        # Upewnij się, że pętla asyncio jest uruchomiona tylko raz
        asyncio.run(analyzer.start())
    except KeyboardInterrupt:
        analyzer.stop()
    except Exception as e:
        logger.critical(f"Krytyczny błąd aplikacji: {e}")
        console.print(f"[bold red]Krytyczny błąd aplikacji: {e}[/bold red]")
    finally:
        # Zamknij klienta ccxt, jeśli istnieje
        if analyzer.exchange:
            asyncio.run(analyzer.exchange.close())
        console.print("[bold blue]Aplikacja zakończona.[/bold blue]")